In [1]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq ollama --progress-bar off
%pip install -qqq pathlib --progress-bar off
%pip install -qqq pandas --progress-bar off
%pip install -qqq PyPDF2 --progress-bar off
%pip install -qqq owlready2 --progress-bar off
%pip install -qqq rdflib --progress-bar off
%pip install -qqq langchain-ollama --progress-bar off
%pip install -qqq langchain-community --progress-bar off
%pip install -qqq langchain_community pypdf --progress-bar off
%pip install -qqq langchain-chroma --progress-bar off
%pip install -qqq rank_bm25  --progress-bar off



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#all the imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import ollama
import json
from enum import Enum
MODEL = "deepseek-r1:8b-llama-distill-q8_0"
import json
import unicodedata
import re

In [5]:
def clean_text(text):
    # Remove LaTeX equations
    text = re.sub(r'\$.*?\$', '', text)  # Remove inline equations
    # Fix hyphenated words
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    # Remove excessive whitespace
    return re.sub(r'\s+', ' ', text).strip()

In [18]:
# Load the PDF
pdf_loader = PyPDFLoader(file_path="LiDAR-Sensors.pdf",extract_images=False)
docs = pdf_loader.load();
# Enhanced cleaning pipeline
for doc in docs:
    doc.page_content = clean_text(doc.page_content)

In [19]:
def analyze_splits(splits):
    stats = {
        'total_chunks': len(splits),
        'avg_chunk_length': sum(len(c.page_content) for c in splits)/len(splits),
        'max_length': max(len(c.page_content) for c in splits),
        'min_length': min(len(c.page_content) for c in splits),
        'metadata_fields': list(splits[0].metadata.keys()) if splits else []
    }
    return stats

In [20]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=lambda text: len(text.split()),  # Word-based counting
        add_start_index=True,
       separators=[
        "\n\n## ",    # Section headers
        "\n\n",       # Paragraph breaks
        "\n",         # New lines
        "(?<!\d)\.(?!\d)\s+",  # Sentence ends with space
        ";",          # Semi-colons
        ", ",         # Commas
        " "
        ],
        keep_separator=True,
        is_separator_regex=True,
    )

splits = text_splitter.split_documents(docs)
analyze_splits(splits)

# word_count(splits)

{'total_chunks': 9,
 'avg_chunk_length': 2127.6666666666665,
 'max_length': 3143,
 'min_length': 435,
 'metadata_fields': ['source', 'page', 'page_label', 'start_index']}

In [21]:
#create embeddings and store in CHROMA database
embeddings = OllamaEmbeddings(model=MODEL)
vector_store = Chroma(
    collection_name="KGLLM",
    embedding_function=embeddings,
)
# Add documents and their embeddings to Chroma (might take some time)
vector_store.add_documents(documents=splits)

# Create individual retrievers
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 3  # Match vector retriever's k

# Create ensemble
retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # Vector search 70%, BM25 30%
)

In [22]:
# Test query
query = "What are the specifications of the LiDAR sensors mentioned?"
results = retriever.get_relevant_documents(query)

# # Inspect results
for doc in results:
    print(doc.metadata)
    print(doc.page_content)


{'page': 1, 'page_label': '25', 'source': 'LiDAR-Sensors.pdf', 'start_index': 2846}
. Removable HDD Ground power supply Fixed-wing or helicopter 200 - 6,000m AGL ~17 hrs @ max pulse rate 0 - 40 °C cabin-side temperature Non-condensing Leica FPES Leica ALS Post Processor for point-cloud generation,T erraScan/T erraModeler for viewing/editing. DEMs, city models, flood plain, erosion & forestery studies, forest floor, disaster management, power-lines, pipe-line and railway and roadway corridors, coastal mapping. Full-function system for a wide variety of applications now features highly integrated FCMS flight-management system and flex - ible external sensor integration. Optech ALTM Gemini October 2006 23.4kg, 26 x 19 x 57cm 95kg, control rack size 65 x 59 x 49cm 28 VDC, 35 A (maximum) 1,060nm 7ns 0.8, 0.25, 0.15 1/e Class IV (FDA 21 CFR) 80 m + Oscillating Mirror 100Hz 167kHz 50 deg 4 ranges Leading edge 12 bits Trimble, 12 channel, dual-frequency 0.05cm POS 610, 200 Hz 0.0025/0.0025/0.0

In [23]:
# Stage 1: Entity-Focused Retrieval
entity_query = """
EXTRACT: LiDAR sensors, their components, technical specifications.
IGNORE: Experimental results, methodology, figures.
"""
entity_docs = retriever.get_relevant_documents(entity_query)

# Stage 2: Relationship Extraction
relation_query = """
FIND: has_part, implements, measurement_properties relationships.
FILTER: Only technical specifications sections.
"""
relation_docs = retriever.get_relevant_documents(relation_query)
# Combine and deduplicate
final_docs = entity_docs + relation_docs
retrieved_text = "\n\n".join([doc.page_content for doc in final_docs])

for doc in final_docs:
    print(doc.metadata)
    print(doc.page_content)

{'page': 18, 'page_label': '19', 'source': 'sensors.pdf', 'start_index': -1}
. Informed Consent Statement: Not applicable. Data Availability Statement: Data available on request due to privacy restrictions. The data presented in this study are available on request from the corresponding authors. The data are not publicly available due to company policies. Conﬂicts of Interest: The authors declare no conﬂict of interest. References 1. Driving, A. Levels of Driving Automation are Deﬁned in New SAE International Standard J3016: 2014; SAE International: Warrendale, PA, USA, 2014. 2. First Internationally Valid System Approval for Conditionally Automated Driving. Daimler. Available online: https: //www.daimler.com/innovation/product-innovation/autonomous-driving/system-approval-for-conditionally-automateddriving.html (accessed on 13 December 2021). 3. Roriz, R.; Cabral, J.; Gomes, T. Automotive LiDAR Technology: A Survey. IEEE Trans. Intell. Transp. Syst. 2022, 23, 6282–6297. doi: 10.1109/T

In [24]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOllama
# Initialize with your local model
llm = ChatOllama(model=MODEL) 
compressor = LLMChainExtractor.from_llm(llm=llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
    search_kwargs={"k": 10}  
)

# Enhanced query
compressed_docs = compression_retriever.invoke(query)

In [29]:
# KG_EXTRACTION_PROMPT = """As a LiDAR sensor expert, analyze this technical text to extract:

# **Target Entities**:
# 1. Sensor Models: Manufacturer-branded names (Velodyne HDL-64E, Livox Horizon)
# 2. Components: Physical/software parts (e.g., MEMS mirror, FPGA processor)
# 3. Technical Specs: Quantified values with units (120m range, 0.08° resolution)
# 4. Implementations: Standards/protocols (IEEE 802.11p, ROS2)

# **Extraction Rules**:
# - Preserve contextual relationships: 
#   "The Velodyne VLP-32C's rotating assembly enables 360° coverage" → 
#   {{"parts": ["rotating assembly"], "properties": ["field of view: 360°"]}}
# - Capture implied properties from comparisons:
#   "Outperforms Ouster OS2 in range" → 
#   {{"properties": ["comparative_range: > Ouster OS2"]}}
# - Retain partial information with "[INFERRED]" tags

# **Enhanced Format**:
# ```json
# {{
#   "sensors": [
#     {{
#       "name": "Livox Horizon",
#       "category": "Automotive Lidar",
#       "parts": ["MEMS mirror", "905nm laser array"],
#       "properties": [
#         "horizontal_fov: 81.7°", 
#         "range: 260m @ 10% reflectivity",
#         "scan_pattern: non-repetitive"
#       ],
#       "implements": ["ROS2", "AutoSAR"]
#     }}
#   ]
# }}
# ```
# <text>
# {text}
# <text>
# """
KG_EXTRACTION_PROMPT="""
As a LiDAR sensor expert, analyze this technical text to extract:

**Target Entities**:
1. Sensor Models: Manufacturer-branded names (Velodyne HDL-64E, Livox Horizon)
2. Components: Physical/software parts (e.g., MEMS mirror, FPGA processor)
3. Technical Specs: Quantified values with units (120m range, 0.08° resolution)
4. Implementations: Standards/protocols (IEEE 802.11p, ROS2)

**Extraction Rules**:
- Preserve contextual relationships: 
  "The Velodyne VLP-32C's rotating assembly enables 360° coverage" → 
  {{"parts": ["rotating assembly"], "properties": ["field of view: 360°"]}}
- Capture implied properties from comparisons:
  "Outperforms Ouster OS2 in range" → 
  {{"properties": ["comparative_range: > Ouster OS2"]}}
- Retain partial information with "[INFERRED]" tags


**Enhanced Format**:
```json
{{
  "sensors": [
    {{
      "name": "Livox Horizon",
      "category": "Automotive Lidar",
      "parts": ["MEMS mirror", "905nm laser array"],
      "properties": [
        "horizontal_fov: 81.7°", 
        "range: 260m @ 10% reflectivity",
        "scan_pattern: non-repetitive"
      ],
      "implements": ["ROS2", "AutoSAR"]
    }}
  ]
}}
```
**Critical Instructions**:
1. If no sensors are found, return: {"sensors": []}
2. Always maintain valid JSON structure
3. Never add extra text outside the JSON
4. Use exact values from tables when available

<example>
Input: "The Velodyne VLP-32C has a 200m range and 0.2° resolution"
Output:
{{
  "sensors": [
    {{
      "name": "Velodyne VLP-32C",
      "category": "Automotive LiDAR",
      "parts": [],
      "properties": ["range: 200m", "resolution: 0.2°"],
      "implements": []
    }}
  ]
}}
</example>


<text>
{text}
<text>

"""

In [30]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"
 
 
def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
    )
    return response["response"]

In [31]:
# Prepare the final prompt with the concatenated text

responses = []

# chunks = [retrieved_text[i:i + CHUNK_SIZE] for i in range(0, len(retrieved_text), CHUNK_SIZE)];
chunks = text_splitter.split_text(retrieved_text)

for i, chunk in enumerate(chunks):
        final_prompt = KG_EXTRACTION_PROMPT.format(text=chunk)
        # Send the final prompt to Ollama
        response = call_model(final_prompt)
        responses.append(response)
print(responses)

KeyError: '"sensors"'

In [28]:
# Save responses after every chunk to ensure progress is retained
with open("R1_responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

In [17]:
import json
import re
import unicodedata

def extract_json_part(response_str):
    """Extracts JSON content from Ollama's response string"""
    try:
        # Get first element of response list
        if isinstance(response_str, list):
            content = response_str[0]
        else:
            content = response_str
            
        # Extract JSON block
        json_str = content.split("```json")[-1].split("```")[0].strip()
        return json.loads(json_str)
    except (IndexError, json.JSONDecodeError) as e:
        print(f"Failed to extract JSON: {str(e)}")
        return None

def clean_string(s):
    """Enhanced cleaning/normalization"""
    if not isinstance(s, str):
        return str(s).lower()
        
    # s = unicodedata.normalize('NFKC', s)
    # s = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', s)  # Remove control chars
    # s = re.sub(r'[_\-\n]+', ' ', s)  # Replace separators with space
    # s = re.sub(r'\s+', ' ', s).strip()
    return s.lower()

# Load responses
with open("R1_responses.json", "r", encoding="utf-8") as f:
    responses = json.load(f)

final_sensors = []
seen_sensors = set()

for response in responses:
    try:
        # Extract JSON from Ollama response format
        response_json = extract_json_part(response)
        if not response_json:
            continue

        # Process sensors with validation
        sensors = response_json.get("sensors", [])
        if not isinstance(sensors, list):
            continue
            
        for sensor in sensors:
            # Validate required fields
            if "name" not in sensor:
                continue
                
            # Clean and normalize
            sensor_name = clean_string(sensor["name"])
            category_name = clean_string(sensor.get("category", "uncategorized"))
            
            # Process lists with null checks
            parts = [clean_string(p) for p in sensor.get("parts", []) if p]
            properties = [clean_string(p) for p in sensor.get("properties", []) if p]
            implements = [clean_string(i) for i in sensor.get("implements", []) if i]

            # Create unique identifier
            sensor_id = hash((
                sensor_name, 
                category_name,
                tuple(sorted(parts)),
                tuple(sorted(properties)),
                tuple(sorted(implements))
            ))

            if sensor_id not in seen_sensors:
                seen_sensors.add(sensor_id)
                final_sensors.append({
                    "name": sensor_name,
                    "category": category_name,
                    "parts": sorted(parts),
                    "properties": sorted(properties),
                    "implements": sorted(implements)
                })
                
    except Exception as e:
        print(f"Error processing entry: {str(e)}")
        continue

# Save results
with open("R1_sensors_merged.json", "w", encoding="utf-8") as out_file:
    json.dump({"sensors": final_sensors}, out_file, ensure_ascii=False, indent=2)

print(f"Processed {len(final_sensors)} unique sensors.")

Processed 11 unique sensors.


In [88]:
from owlready2 import *
import json

# Load the JSON file
with open("R1_sensors_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a new ontology
onto = get_ontology("http://example.org/sensor_ontology.owl")

with onto:
    # Define basic classes and properties
    class Sensor(Thing):
        pass
    
    class Part(Thing):
        pass
    
    class Property(Thing):
        pass
    
    class Technology(Thing):
        pass

    class has_part_directly(ObjectProperty):
        domain = [Sensor]
        range = [Part]
    
    class implements(ObjectProperty):
        domain = [Sensor]
        range = [Technology]
    
    class has_property(ObjectProperty):
        domain = [Sensor]
        range = [Property]

    # Define an annotation property for category
    class category(AnnotationProperty):
        pass

    # Process the JSON data
    part_classes = {}  # Cache for part classes to avoid duplicates
    property_classes = {}  # Cache for property classes
    technology_classes = {}  # Cache for technology classes

    for sensor_data in data["sensors"]:
        # Create or get the sensor class
        sensor_name = sensor_data["name"].replace(" ", "_")
        sensor_category = sensor_data["category"].replace(" ", "_")
        
        # Define the sensor class dynamically
        sensor_class = types.new_class(sensor_name, (Sensor,))
        
        # Assign category as an annotation
        sensor_class.category.append(sensor_category)

        # Add parts and relationships
        for part_name in sensor_data["parts"]:
            part_name_cleaned = part_name.replace(" ", "_")
            if part_name_cleaned not in part_classes:
                part_class = types.new_class(part_name_cleaned, (Part,))
                part_classes[part_name_cleaned] = part_class
            else:
                part_class = part_classes[part_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(has_part_directly.some(part_class))

        # Add properties and relationships
        for property_name in sensor_data["properties"]:
            property_name_cleaned = property_name.replace(" ", "_")
            if property_name_cleaned not in property_classes:
                property_class = types.new_class(property_name_cleaned, (Property,))
                property_classes[property_name_cleaned] = property_class
            else:
                property_class = property_classes[property_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(has_property.some(property_class))

        # Add technologies (implements relationship)
        for tech_name in sensor_data["implements"]:
            tech_name_cleaned = tech_name.replace(" ", "_")
            if tech_name_cleaned not in technology_classes:
                tech_class = types.new_class(tech_name_cleaned, (Technology,))
                technology_classes[tech_name_cleaned] = tech_class
            else:
                tech_class = technology_classes[tech_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(implements.some(tech_class))

# Save the ontology to a file
onto.save(file="R1_sensor_ontology_with_properties_and_implements.owl", format="rdfxml")

print("Ontology created and saved as sensor_ontology_with_properties_and_implements.owl")


Ontology created and saved as sensor_ontology_with_properties_and_implements.owl
